In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

In [2]:
df = pd.read_csv("Virat-Kohli-International-Cricket-Centuries.csv")

In [3]:
# Display the DataFrame without the index with (.to_string(index = False))

# print(df.to_string(index=False))                          # full data without automated serial number
# print(df.drop(columns=['Against']))                       # Full data without the selected column
# print(df.iloc[70:80].to_string(index=False))              # data required from a specific range
# print(tabulate(df, headers='keys', tablefmt='grid'))      # to print data in table format

Classification

In [4]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [5]:
df.info()
print("--------------")
df.describe()

# df_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   No.       80 non-null     int64 
 1   Runs      80 non-null     int64 
 2   Against   80 non-null     object
 3   Position  80 non-null     int64 
 4   Innings   80 non-null     int64 
 5   Venue     80 non-null     object
 6   Ground    80 non-null     object
 7   Date      80 non-null     object
 8   Result    80 non-null     object
dtypes: int64(4), object(5)
memory usage: 5.8+ KB
--------------


,No.,Runs,Position,Innings
count,80.0000,80.000000,80.000000,80.00000
mean,40.5000,131.550000,3.500000,1.67500
std,23.2379,35.094772,0.656033,0.67082
min,1.0000,100.000000,2.000000,1.00000
25%,20.7500,107.000000,3.000000,1.00000
50%,40.5000,118.500000,3.000000,2.00000
75%,60.2500,139.250000,4.000000,2.00000
max,80.0000,254.000000,6.000000,4.00000


In [6]:
df_class = pd.get_dummies(df, drop_first=False)
train, valid ,test = np.split(df_class.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])            #after np.split if df.sample(frac=1)  then it will shuffle data and only df will choose data linewise for division in train, test and valid.


C:\Users\1\AppData\Roaming\Python\Python312\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
df_class.info()
print("--------------")

df.describe()
print("--------------")


df_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Columns: 150 entries, No. to Result_Won
dtypes: bool(146), int64(4)
memory usage: 14.0 KB
--------------
--------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Columns: 150 entries, No. to Result_Won
dtypes: bool(146), int64(4)
memory usage: 14.0 KB


In [8]:
print(train.dtypes)

No.                     int64
Runs                    int64
Position                int64
Innings                 int64
Against_ Afghanistan     bool
                        ...  
Date_31-Jul-12           bool
Result_Drawn             bool
Result_Lost              bool
Result_Tied              bool
Result_Won               bool
Length: 150, dtype: object


In [9]:
def scale_dataset(dataframe, oversample=True) :

  x = dataframe.iloc[:, :-1].values  
  y = dataframe.iloc[:, -1].values

  scaler = StandardScaler()
  x = scaler.fit_transform(x)

  if oversample:
        ros = RandomOverSampler(random_state=42)
        x, y = ros.fit_resample(x, y)

  return np.hstack((x, np.reshape(y, (-1, 1)))), x, y

In [10]:
print(train)


    No.  Runs  Position  Innings  Against_ Afghanistan  Against_ Australia  \
67   68   114         3        2                 False               False   
61   62   107         3        2                 False               False   
48   49   113         3        1                 False               False   
34   35   117         3        1                 False                True   
4     5   100         4        1                 False               False   
62   63   123         4        2                 False                True   
2     3   118         3        2                 False                True   
58   59   139         4        1                 False               False   
69   70   136         4        2                 False               False   
12   13   106         3        1                 False               False   
79   80   117         3        1                 False               False   
76   77   122         3        1                 False          

In [11]:
train, x_train, y_train = scale_dataset(train, oversample=True)
valid, x_valid, y_valid = scale_dataset(valid, oversample=False)
test, x_test, y_test = scale_dataset(test, oversample=False)

In [12]:
print("Unique classes in y_train:", np.unique(y_train))
if len(np.unique(y_train)) <= 1:
    print("Warning: y_train contains less than 2 classes.")
    # Handle this case (e.g., skip this iteration or apply balancing techniques)


Unique classes in y_train: [False  True]


In [13]:
#kNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

knn_model = KNeighborsClassifier(n_neighbors=10)
knn_model.fit(x_train, y_train)
y_pred = knn_model.predict(x_test)

print("KNN :")
print(classification_report(y_test, y_pred))


KNN :
              precision    recall  f1-score   support

       False       1.00      0.33      0.50         6
        True       0.71      1.00      0.83        10

    accuracy                           0.75        16
   macro avg       0.86      0.67      0.67        16
weighted avg       0.82      0.75      0.71        16



In [14]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model = nb_model.fit(x_train, y_train)
y_pred = nb_model.predict(x_test)
print("Naive Bayes :")
print(classification_report(y_test, y_pred))

Naive Bayes :
              precision    recall  f1-score   support

       False       0.80      0.67      0.73         6
        True       0.82      0.90      0.86        10

    accuracy                           0.81        16
   macro avg       0.81      0.78      0.79        16
weighted avg       0.81      0.81      0.81        16



In [15]:
print("Unique classes in y_train:", np.unique(y_train))
print("Class counts in y_train:", np.bincount(y_train))

Unique classes in y_train: [False  True]
Class counts in y_train: [34 34]


In [16]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lg_model = LogisticRegression(max_iter=80)  # Increase max iterations if needed
lg_model.fit(x_train, y_train)
y_pred = lg_model.predict(x_test)

print("Logistic Regression:")
print(classification_report(y_test, y_pred))


Logistic Regression:
              precision    recall  f1-score   support

       False       1.00      0.33      0.50         6
        True       0.71      1.00      0.83        10

    accuracy                           0.75        16
   macro avg       0.86      0.67      0.67        16
weighted avg       0.82      0.75      0.71        16



In [17]:
#SVM
from sklearn.svm import SVC

svm_model = SVC()
svm_model = svm_model.fit(x_train, y_train)
y_pred = svm_model.predict(x_test)

print("SVM :")
print(classification_report(y_test, y_pred))



SVM :
              precision    recall  f1-score   support

       False       1.00      0.17      0.29         6
        True       0.67      1.00      0.80        10

    accuracy                           0.69        16
   macro avg       0.83      0.58      0.54        16
weighted avg       0.79      0.69      0.61        16

